In [1]:
from typing_extensions import Self
from dataclasses import dataclass
from enum import Enum

In [2]:
# Load the input
with open("day7_input.txt") as f:
    day7_input = f.read()

In [3]:
example_input = """
32T3K 765
T55J5 684
KK677 28
KTJJT 220
QQQJA 483
""".strip()

In [4]:
# Helper classes
# Card enum
class Card(Enum):
    TWO = 0
    THREE = 1
    FOUR = 2
    FIVE = 3
    SIX = 4
    SEVEN = 5
    EIGHT = 6
    NINE = 7
    TEN = 8
    JACK = 9
    QUEEN = 10
    KING = 11
    ACE = 12

# Map strings to card types
CHAR_TO_CARD_MAP = {
    "2": Card.TWO,
    "3": Card.THREE,
    "4": Card.FOUR,
    "5": Card.FIVE,
    "6": Card.SIX,
    "7": Card.SEVEN,
    "8": Card.EIGHT,
    "9": Card.NINE,
    "T": Card.TEN,
    "J": Card.JACK,
    "Q": Card.QUEEN,
    "K": Card.KING,
    "A": Card.ACE,
}

# Hand type enum
class HandType(Enum):
    HIGH_CARD = 0
    ONE_PAIR = 1
    TWO_PAIR = 2
    THREE_OAK = 3
    FULL_HOUSE = 4
    FOUR_OAK = 5
    FIVE_OAK = 6

    @staticmethod
    def GetHandTypeFromCards(cards: list[Card]) -> Self:
        # Really basic approach, count how many of each type of card there is and manually implement
        # the reasoning based on how many pairs / triplets etc. there are
        card_occurrences = {card: 0 for card in Card}
        for card in cards:
            card_occurrences[card] = card_occurrences[card] + 1
        
        # Count up how many pairs / triples there are.
        # If we find a quadruple / quintent we can return straight away
        pair_count = trip_count = 0
        for occurrences in card_occurrences.values():
            if occurrences == 5:
                return HandType.FIVE_OAK
            elif occurrences == 4:
                return HandType.FOUR_OAK
            elif occurrences == 3:
                trip_count += 1
            elif occurrences == 2:
                pair_count += 1
        
        # Trip is either full house or three of a kind
        if trip_count == 1: return HandType.FULL_HOUSE if pair_count == 1 else HandType.THREE_OAK

        # Now just two pair / one pair / high card left
        return HandType.TWO_PAIR if pair_count == 2 else HandType.ONE_PAIR if pair_count == 1 else HandType.HIGH_CARD

# Hand class
@dataclass
class Hand:
    cards: list[Card]
    hand_type: HandType
    bid: int
    _hand_value: int = -1 # Invalid

    # Returns a key value for use in sorting.
    @property
    def sort_key(self: Self) -> int:
        return self._hand_value

    # Calculate the hand value after construction
    # Going to treat this as turning each hand into a 6 digit base-13 number. The most significant
    # digit is the hand type * 13^5, then the first card value * 13^4 etc. down to last card * 13^0
    def __post_init__(self):
        cards_value = 0
        for i in range(5):
            cards_value += self.cards[i].value * 13**(4-i)

        self._hand_value = self.hand_type.value * 13**5 + cards_value

    @staticmethod
    def Parse(hand_str: str) -> Self:
        cards_str, bid_str = hand_str.split()
        assert(len(cards_str) == 5)
        cards = [CHAR_TO_CARD_MAP[char] for char in cards_str]
        hand_type = HandType.GetHandTypeFromCards(cards)
        return Hand(cards=cards, hand_type=hand_type, bid=int(bid_str))

In [5]:
## Class testing

# HandType tests
hand_type_tests = [
    {
        "cards": [Card.THREE , Card.TWO   , Card.TEN   , Card.THREE , Card.KING ],
        "expected_hand_type": HandType.ONE_PAIR,
    },
    {
        "cards": [Card.TEN   , Card.FIVE  , Card.FIVE  , Card.JACK  , Card.FIVE ],
        "expected_hand_type": HandType.THREE_OAK,
    },
    {
        "cards": [Card.KING  , Card.KING  , Card.SIX   , Card.SEVEN , Card.SEVEN],
        "expected_hand_type": HandType.TWO_PAIR,
    },
    {
        "cards": [Card.KING  , Card.TEN   , Card.JACK  , Card.JACK  , Card.TEN  ],
        "expected_hand_type": HandType.TWO_PAIR,
    },
    {
        "cards": [Card.QUEEN , Card.QUEEN , Card.QUEEN , Card.JACK  , Card.ACE  ],
        "expected_hand_type": HandType.THREE_OAK,
    },
]
for ht_test in hand_type_tests:
    output_hand_type = HandType.GetHandTypeFromCards(ht_test["cards"])
    expected_hand_type = ht_test["expected_hand_type"]
    pass_str = "PASS" if expected_hand_type == output_hand_type else "FAIL"
    print(f"HandType: {pass_str}. Expected: {expected_hand_type}. Actual: {output_hand_type}.")


# Hand parsing tests
print()
expected_hands = [
    Hand(cards=[Card.THREE , Card.TWO   , Card.TEN   , Card.THREE , Card.KING ], hand_type=HandType.ONE_PAIR  , bid=765),
    Hand(cards=[Card.TEN   , Card.FIVE  , Card.FIVE  , Card.JACK  , Card.FIVE ], hand_type=HandType.THREE_OAK , bid=684),
    Hand(cards=[Card.KING  , Card.KING  , Card.SIX   , Card.SEVEN , Card.SEVEN], hand_type=HandType.TWO_PAIR  , bid=28),
    Hand(cards=[Card.KING  , Card.TEN   , Card.JACK  , Card.JACK  , Card.TEN  ], hand_type=HandType.TWO_PAIR  , bid=220),
    Hand(cards=[Card.QUEEN , Card.QUEEN , Card.QUEEN , Card.JACK  , Card.ACE  ], hand_type=HandType.THREE_OAK , bid=483),
]
example_hands = [Hand.Parse(hand_str=line) for line in example_input.split("\n")]

for i in range(len(example_hands)):
    pass_str = "PASS" if expected_hands[i] == example_hands[i] else "FAIL"
    print(f"Hand parsing: {pass_str}. Expected: {expected_hands[i]}. Actual: {example_hands[i]}.")

# Hand value tests
print()
hand_value_tests = [
    {
        "hand": Hand(cards=[Card.KING  , Card.KING  , Card.SIX   , Card.SEVEN , Card.SEVEN], hand_type=HandType.TWO_PAIR  , bid=28),
        "expected_value": 1081670,
    },
    {
        "hand": Hand(cards=[Card.ACE   , Card.ACE   , Card.ACE   , Card.ACE   , Card.ACE  ], hand_type=HandType.FIVE_OAK  , bid=0),
        "expected_value": 2599050,
    },
]
for hv_test in hand_value_tests:
    output_value = hv_test["hand"].sort_key
    expected_value = hv_test["expected_value"]
    pass_str = "PASS" if expected_value == output_value else "FAIL"
    print(f"Hand value: {pass_str}. Expected: {expected_value}. Actual: {output_value}.")

# Hand ranking test
print()
expected_hand_order = [
    Hand(cards=[Card.THREE , Card.TWO   , Card.TEN   , Card.THREE , Card.KING ], hand_type=HandType.ONE_PAIR  , bid=765),
    Hand(cards=[Card.KING  , Card.TEN   , Card.JACK  , Card.JACK  , Card.TEN  ], hand_type=HandType.TWO_PAIR  , bid=220),
    Hand(cards=[Card.KING  , Card.KING  , Card.SIX   , Card.SEVEN , Card.SEVEN], hand_type=HandType.TWO_PAIR  , bid=28),
    Hand(cards=[Card.TEN   , Card.FIVE  , Card.FIVE  , Card.JACK  , Card.FIVE ], hand_type=HandType.THREE_OAK , bid=684),
    Hand(cards=[Card.QUEEN , Card.QUEEN , Card.QUEEN , Card.JACK  , Card.ACE  ], hand_type=HandType.THREE_OAK , bid=483),
]
example_hands.sort(key=lambda hand: hand.sort_key)
for i in range(len(expected_hand_order)):
    expected_hand = expected_hand_order[i]
    actual_hand = example_hands[i]
    pass_str = "PASS" if expected_hand == actual_hand else "FAIL"
    print(f"Hand rank {i+1}: {pass_str}. Expected: {expected_hand}. Actual: {actual_hand}.")

HandType: PASS. Expected: HandType.ONE_PAIR. Actual: HandType.ONE_PAIR.
HandType: PASS. Expected: HandType.THREE_OAK. Actual: HandType.THREE_OAK.
HandType: PASS. Expected: HandType.TWO_PAIR. Actual: HandType.TWO_PAIR.
HandType: PASS. Expected: HandType.TWO_PAIR. Actual: HandType.TWO_PAIR.
HandType: PASS. Expected: HandType.THREE_OAK. Actual: HandType.THREE_OAK.

Hand parsing: PASS. Expected: Hand(cards=[<Card.THREE: 1>, <Card.TWO: 0>, <Card.TEN: 8>, <Card.THREE: 1>, <Card.KING: 11>], hand_type=<HandType.ONE_PAIR: 1>, bid=765, _hand_value=401230). Actual: Hand(cards=[<Card.THREE: 1>, <Card.TWO: 0>, <Card.TEN: 8>, <Card.THREE: 1>, <Card.KING: 11>], hand_type=<HandType.ONE_PAIR: 1>, bid=765, _hand_value=401230).
Hand parsing: PASS. Expected: Hand(cards=[<Card.TEN: 8>, <Card.FIVE: 3>, <Card.FIVE: 3>, <Card.JACK: 9>, <Card.FIVE: 3>], hand_type=<HandType.THREE_OAK: 3>, bid=684, _hand_value=1349585). Actual: Hand(cards=[<Card.TEN: 8>, <Card.FIVE: 3>, <Card.FIVE: 3>, <Card.JACK: 9>, <Card.FIVE

In [6]:
# Get the winnings from some hands
def get_winnings(hands: list[Hand]) -> int:
    # Rank the hands, this puts the strongest hand last which is what we want
    hands.sort(key=lambda hand: hand.sort_key, reverse=False)
    winnings = 0
    for i, hand in enumerate(hands):
        rank = i + 1
        winnings += hand.bid * rank

    return winnings

In [7]:
# Test winnings calculation
expected_winnings = 6440
example_winnings = get_winnings([Hand.Parse(hand_str=line) for line in example_input.split("\n")])
pass_str = "PASS" if expected_winnings == example_winnings else "FAIL"
print(f"Winnings test: {pass_str}. Expected: {expected_winnings}. Actual: {example_winnings}.")


Winnings test: PASS. Expected: 6440. Actual: 6440.


In [8]:
# Parse the actual input
part_one_hands = [Hand.Parse(hand_str=line) for line in day7_input.split("\n")]

In [ ]:
print(f"Answer (part one): {get_winnings(part_one_hands)}")